In [7]:
#import tensorflow backend
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import keras
from keras.layers import *
from keras.models import Sequential, Model
from keras.optimizers import *
from keras.preprocessing import sequence
print(keras.__version__)
print(keras.backend.backend())
import numpy as np

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7388235101743159890
]


Using TensorFlow backend.


2.1.2
tensorflow


In [4]:
#load data
with open("temp/neg_sample_data.json", "r", encoding="utf-8") as f:
    md_nfreq_sample = json.load(f)
    
with open("temp/tweets4classification.json", "r", encoding="utf-8") as f:
    md_freq_sample = json.load(f)

In [5]:
md_nfreq_sample['data'][0]

{'hashtag_label': [],
 'hashtags': ['cervicalhealthawarenessmonth', 'obamacar'],
 'id': '418769437261455360',
 'orignal_hashtags': ['#cervicalhealthawarenessmonth', '#obamacare'],
 'raw': 'rt @ppsne: jan. is #cervicalhealthawarenessmonth. thanks to #obamacare, pap tests + the hpv vaccine are covered without a copay. http://t.c…',
 'text': 'rt : jan. is cervicalhealthawarenessmonth. thanks to obamacare, pap tests + the hpv vaccine are covered without a copay.',
 'words': ['rt',
  ':',
  'jan.',
  'is',
  'cervicalhealthawarenessmonth',
  '.',
  'thanks',
  'to',
  'obamacare',
  ',',
  'pap',
  'tests',
  '+',
  'the',
  'hpv',
  'vaccine',
  'are',
  'covered',
  'without',
  'a',
  'copay',
  '.']}

In [6]:
md_freq_sample['data'][0]

{'hashtag_label': [1, 5],
 'hashtags': ['hpv', 'vaccin'],
 'id': '418263863772327936',
 'orignal_hashtags': ['#hpv', '#vaccine'],
 'raw': 'rt @cdcstd: #hpv vax coverage could be 93% if doctors gave hpv #vaccine each time a preteen/teen got any other vaccine&gt; http://t.co/xxryga5…',
 'text': 'rt : hpv vax coverage could be 93% if doctors gave hpv vaccine each time a preteen / teen got any other vaccine>',
 'words': ['rt',
  ':',
  'hpv',
  'vax',
  'coverage',
  'could',
  'be',
  '93',
  '%',
  'if',
  'doctors',
  'gave',
  'hpv',
  'vaccine',
  'each',
  'time',
  'a',
  'preteen',
  '/',
  'teen',
  'got',
  'any',
  'other',
  'vaccine',
  '>']}